## Assignment 3

#### Import Libraries

In [7]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import csv
import chess
import chess.pgn
import io
import re
import time
# import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

#### Import CSV file

66879 entries in dataframe

In [8]:
# Load CSV into a big dataframe
CHESS_DATA_LOCATION = "data/club_games_data.csv"
chess_data = pd.read_csv(CHESS_DATA_LOCATION)

#### Dataframe info

Using:
print(chess_data.dtypes)
print(chess["rules"].unique())


Chess rules:
['chess' 'chess960' 'threecheck' 'crazyhouse' 'kingofthehill']

Time control:
['1/259200' '1/172800' '1800' '1/86400' '1/432000' '1/604800' '600'
 '120+1' '900+10' '300' '180+2' '3600+5' '2700+45' '3600' '1/1209600'
 '180' '600+10' '60' '480+3' '300+5' '420+3' '600+5' '600+2' '1200' '30'
 '60+1' '120' '1500+3' '900+2' '1500+5' '1500+10' '1/864000' '900' '300+2'
 '1500' '7200' '300+1' '5400' '3600+60' '2700+30' '3480+45' '10' '2700+10'
 '15' '2700' '3600+20' '4500' '4200' '900+5' '1800+10' '2700+5' '480+5'
 '1800+30' '300+3' '600+1' '1800+5' '420+5' '5400+30' '240+10' '420' '303'
 '60+10']

 Time class:
['daily' 'rapid' 'bullet' 'blitz']

#### Clean data

Undeveloped board shouldn't matter if we're filtering games for potential ep
Same for draws

In [9]:
# Drops rows if any value is a NaN (data is clean so it doesn't do anything)
chess_data.dropna(axis = 0, how = 'any')

# Filter out alternative rules like chess960 etc
chess_data = chess_data[chess_data['rules'] == "chess"]


#### Stratified sample
Is this needed for Q2?
Commented for now

In [10]:
# print(len(chess_data[chess_data["time_class"] == "daily"]))
# print(len(chess_data[chess_data["time_class"] == "rapid"]))
# print(len(chess_data[chess_data["time_class"] == "bullet"]))
# print(len(chess_data[chess_data["time_class"] == "blitz"]))

# # Select stratified sample (grouping proportionally by time class)
# chess_data_sample = chess_data.groupby('time_class', group_keys=False).apply(lambda x: x.sample(frac=0.2))

#### Get pgn from last column

In [11]:
# Save pgn column from dataframe
full_pgn = chess_data['pgn']

def get_moves(entry):
    '''
    Retrive series of moves in a game when given the whole full_pgn entry
    '''
    # # Find last occurrence of ']' in each full_pgn entry
    # link_bracket = entry.rindex('[Link ')

    # # Cut non-move info of full_pgn
    # pgn = full_pgn[link_bracket ::]

    # # Find first occurrence of '1. ', signifying first move, in each full_pgn entry
    # first_move_index = pgn.rindex('1. ')


    # pgn is just a string "separated" by newlines so turn into an array, and pgn is
    # always the final entry in the array. though turning into array may not be the most
    # efficient maybe? not sure tho  - leon
    pgn = entry.splitlines()[-1]
    return pgn

chess_data['move_list'] = full_pgn.apply(get_moves)
print(chess_data['move_list'][134])
# get_moves(full_pgn[0])
# ouput: '1. d4 Nf6 2. c4 e6 3. Nc3 Bb4 4. Qb3 Bxc3+ 5. Qxc3 O-O 6. Bg5 c5 7. dxc5 Nc6 8. Nf3 Qa5 9. Bxf6 gxf6 10. Qxa5 Nxa5 11. e3 Rd8 12. Rd1 Kg7 13. Be2 b6 14. Rd4 bxc5 15. Rg4+ Kh6 16. Bd3 f5 17. Rh4+ Kg6 18. g4 Ba6 19. gxf5+ exf5 20. Ne5+ Kf6 21. Rh6+ Kxe5 22. f4# 1-0'

1. d4 {[%clk 0:09:48]} 1... Nf6 {[%clk 0:09:58.3]} 2. Nc3 {[%clk 0:09:45.4]} 2... d5 {[%clk 0:09:57.2]} 3. e4 {[%clk 0:09:44.5]} 3... dxe4 {[%clk 0:09:55.3]} 4. f3 {[%clk 0:09:43.5]} 4... Nc6 {[%clk 0:09:45.5]} 5. fxe4 {[%clk 0:09:37]} 5... Qxd4 {[%clk 0:09:42.5]} 6. Qxd4 {[%clk 0:09:34]} 6... Nxd4 {[%clk 0:09:41.5]} 7. Bb5+ {[%clk 0:09:21.6]} 7... c6 {[%clk 0:09:37.4]} 8. Ba4 {[%clk 0:09:20.6]} 8... b5 {[%clk 0:09:35.6]} 9. Bb3 {[%clk 0:09:19.2]} 9... b4 {[%clk 0:09:32.9]} 10. Na4 {[%clk 0:09:10.2]} 10... e5 {[%clk 0:09:24.7]} 11. Be3 {[%clk 0:09:05.3]} 11... Nxb3 {[%clk 0:09:12.3]} 12. cxb3 {[%clk 0:08:56.7]} 12... Nxe4 {[%clk 0:09:10.4]} 13. Nf3 {[%clk 0:08:48.1]} 13... Bg4 {[%clk 0:09:00.7]} 14. O-O {[%clk 0:08:45.3]} 14... Be7 {[%clk 0:08:54.4]} 15. Nxe5 {[%clk 0:08:40.4]} 15... Be2 {[%clk 0:08:41.1]} 16. Rxf7 {[%clk 0:08:25.6]} 16... Bf6 {[%clk 0:08:20.3]} 17. Bf4 {[%clk 0:08:01.1]} 17... Bxe5 {[%clk 0:08:17.8]} 18. Rf5 {[%clk 0:07:50.3]} 18... Bd4+ {[%clk 0:08:04.6]} 19. Kh1 {[%

#### Import chess info
https://python-chess.readthedocs.io/en/latest/core.html#chess.Board.san


#### En Passant functions
- has_legal_en_passant() tests if en passant capturing would actually be possible on the next move.
- has_pseudo_legal_en_passant()
- has_legal_en_passant()
- is_en_passant(move: Move) Checks if the given pseudo-legal move is an en passant capture.




Use StringIO to parse games from a string.

```python
import io
pgn = io.StringIO("1. e4 e5 2. Nf3 *")
game = chess.pgn.read_game(pgn)
```

### Number of games with at least 1 possible en-passant move:
## 4750
(maybe 4913 or even 4945)

### Number of games with an actual en-passant move:
## 1566

### Sample for Q2 and apply PCA

Should we also filter data so that each player appears once? Idk

In [ ]:
CHESS_DATA_LOCATION = "data/club_games_data.csv"
# MOVIES_DATA_LOCATION = "data/movies.csv"

# load csv into a big dataframe
chess_data = pd.read_csv(CHESS_DATA_LOCATION)

# Drop irrelevant columns
chess_data = chess_data.drop(['white_id', 'black_id'], axis=1)

print(chess_data.head(5))

  white_username black_username  white_rating  black_rating white_result  \
0         -Amos-    miniman2804          1708          1608          win   
1         -Amos-      koltcho69          1726          1577          win   
2         -Amos-      enhmandah          1727           842          win   
3      enhmandah         -Amos-           819          1727   checkmated   
4         -Amos-  Shalllow-Blue          1729          1116          win   

  black_result time_class time_control  rules  rated  \
0   checkmated      daily     1/259200  chess   True   
1     resigned      daily     1/172800  chess   True   
2     resigned      daily     1/172800  chess   True   
3          win      daily     1/172800  chess   True   
4     resigned      daily     1/172800  chess   True   

                                                 fen  \
0  r2r4/p2p1p1p/b6R/n1p1kp2/2P2P2/3BP3/PP5P/4K2R ...   
1       8/5Q1k/4n1pp/8/7P/2N2b2/PP3P2/5K2 b - - 1 33   
2  rn1q1b1r/kb2p1pp/2p5/p1Q5/N1BP2n1/4